In [ ]:
!pip install ../input/wheat-detection/yacs-0.1.7-py3-none-any.whl
!pip install ../input/wheat-detection/torch-1.5.0cu101-cp37-cp37m-linux_x86_64.whl
!pip install ../input/wheat-detection/fvcore-0.1.1.post20200716-py3-none-any.whl
!pip install ../input/wheat-detection/pycocotools-2.0.1-cp37-cp37m-linux_x86_64.whl
!pip install ../input/wheat-detection/torchvision-0.6.0cu101-cp37-cp37m-linux_x86_64.whl
!pip install ../input/wheat-detection/detectron2-0.2cu101-cp37-cp37m-linux_x86_64.whl

In [ ]:
import detectron2
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2 import model_zoo
from detectron2.utils.visualizer import Visualizer


import matplotlib.pyplot as plt
import cv2
import pandas as pd

from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
cfg = get_cfg() # initialize cfg object
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))  # load default parameters for Mask R-CNN
cfg.MODEL.DEVICE='cuda'  # 'cpu' to force model to run on cpu, 'cuda' if you have a compatible gpu
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.WEIGHTS = '../input/model-25000/model_final.pth'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.25
predictor = DefaultPredictor(cfg)

In [ ]:
import ast
import json as json



df = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv")


coco = {
    "info": {
        "year": "2021",
        "version": "1",
        "description": "Help Protect the Great Barrier Reef",
        "contributor": "Tensorflow",
        "url": "",
        "date_created": "2021-11-29T19:36:39+00:00"
    },
    "licenses": [
        {
            "id": 1,
            "url": "https://creativecommons.org/publicdomain/zero/1.0/",
            "name": "Public Domain"
        }
    ],
    "categories": [
        {
            "id": 1,
            "name": "class 1"}
    ],
    "images": [],
    "annotations": []
}

df_2 = df[df.video_id==2]
for i, anotation in enumerate(df_2.annotations):
    if len(anotation)>2:
        coco["images"].append({"id": str(df_2.iloc[i].image_id),
                               "license": 1,
                               "file_name": "video_2/" + str(df_2.iloc[i].video_frame) + ".jpg" ,
                               "height": 720,
                               "width":1280
                               })
        boxes = ast.literal_eval(df_2.iloc[i].annotations)
        for j, box in enumerate(boxes):
            bbox = list(box.values())
            coco["annotations"].append({
                "id": str(df_2.iloc[i].image_id).replace("-", "") + "2" + str(j),
                "area": bbox[2] * bbox[3],
                "image_id": str(df_2.iloc[i].image_id),
                "category_id": 1,
                "bbox": bbox,
                "segmentation": [],
                "iscrowd": 0
            })

with open('test_dataset_coco.json', 'w') as f:
    json.dump(coco, f)

In [ ]:
def format_prediction(bboxes, confs):
    annot = ''
    if len(bboxes)>0:
        for idx in range(len(bboxes)):
            xmin, ymin, xmax, ymax = bboxes[idx]
            w=xmax-xmin
            h=ymax-ymin
            conf             = confs[idx]
            annot += f'{conf} {xmin} {ymin} {w} {h}'
            annot +=' '
        annot = annot.strip(' ')
    return annot

In [ ]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data.catalog import DatasetCatalog

DatasetCatalog.clear()

register_coco_instances("coco_test_dataset2", {}, "test_dataset_coco.json" ,   "../input/tensorflow-great-barrier-reef/train_images")


In [ ]:
from detectron2.data import MetadataCatalog
dataset_test_metadata = MetadataCatalog.get("coco_test_dataset2")
dataset_test = DatasetCatalog.get("coco_test_dataset2")

In [ ]:
def show_img(img, bboxes, bbox_format='yolo'):
    names  = ['starfish']*len(bboxes)
    labels = [0]*len(bboxes)
    img    = draw_bboxes(img = img,
                           bboxes = bboxes, 
                           classes = names,
                           class_ids = labels,
                           class_name = True, 
                           colors = colors, 
                           bbox_format = bbox_format,
                           line_thickness = 2)
    return Image.fromarray(img).resize((800, 400))

In [ ]:
import random

random_testing = random.sample(dataset_test, 20)

for i in range(20):
    print(i)
    fig, ax = plt.subplots(2, 1, figsize =(40,20))
    img = cv2.imread(random_testing[i]['file_name'])

    visualizer = Visualizer(img[:, :, ::-1],
                          metadata=dataset_test_metadata,
                          scale=0.5)
    out = visualizer.draw_dataset_dict(random_testing[i])
    ax[0].imshow(cv2.cvtColor(out.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    ax[0].set_title("Imagen original")
    outputs = predictor(img)
    visualizer_pred = Visualizer(img[:, :, ::-1],
                              metadata=dataset_test_metadata,
                              scale=0.5)
    pred = visualizer_pred.draw_instance_predictions(outputs['instances'].to('cpu'))
  
    ax[1].imshow(cv2.cvtColor(pred.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    ax[1].set_title("Predicción")
    plt.show()

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize =(100,80))
for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
    outputs = predictor(img[:,:,::-1])
    pred_boxes = outputs['instances'].pred_boxes.tensor
    pred_scores =outputs['instances'].scores
    pred_boxes = pred_boxes.cpu().numpy()
    pred_scores = pred_scores.cpu().numpy()
    annot = format_prediction(pred_boxes,pred_scores)
    pred_df['annotations'] = annot
    env.predict(pred_df)
    if idx<3:
        visualizer_pred = Visualizer(img[:, :, ::-1],   
                                 scale=0.5)
        pred = visualizer_pred.draw_instance_predictions(outputs['instances'].to('cpu'))
        ax.grid(False)
        ax.axis('off')
        ax.imshow(cv2.cvtColor(pred.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))